In [2]:
#!nvidia-smi

In [ ]:
#!pip install ultralytics motrackers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 112.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.5/161.5 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
from ultralytics import YOLO
from motrackers import CentroidTracker
import cv2
import numpy as np
import scipy.spatial as spatial
import time

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
main_path = "/content/gdrive/MyDrive/Chicken_Health and_Behavior_Detection"
destination_path = '/content'

In [7]:
!cp "$main_path/models/object_detection/model_yolov11s.zip" "$destination_path"

In [8]:
!unzip -q "$destination_path/model_yolov11s.zip" -d "$destination_path/model"

In [9]:
model = YOLO(f'{destination_path}/model/yolov11s.pt')

In [10]:
video_path = f'{main_path}/data/processed/video_test.mp4'
output_path = f'{destination_path}/output_video_test_tw_density.avi'

In [11]:
#Open Video
cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Video writer
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'XVID'), fps, (frame_width, frame_height))

# Tracker
tracker = CentroidTracker(max_lost=5, tracker_output_format='mot_challenge')
track_data = {}

In [12]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    start_time = time.time()

    # Run YOLO inference
    results = model(frame)

    # Extract bounding boxes
    detections = []
    confidences = []
    class_ids = []

    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()
        scores = result.boxes.conf.cpu().numpy()
        classes = result.boxes.cls.cpu().numpy()

        for i, box in enumerate(boxes):
            x1, y1, x2, y2 = map(int, box)
            w, h = x2 - x1, y2 - y1
            detections.append([x1, y1, w, h])
            confidences.append(scores[i])
            class_ids.append(classes[i])

    # Update tracker
    tracks = tracker.update(np.array(detections), np.array(confidences), np.array(class_ids))

    # List to store current centroids for density analysis
    all_centroids = []

    # Draw tracks and calculate distances
    for trk in tracks:
        track_id = int(trk[1])
        xmin, ymin, w, h = map(int, trk[2:6])
        cx, cy = xmin + w // 2, ymin + h // 2
        all_centroids.append((cx, cy))

        # Initialize data if new ID
        if track_id not in track_data:
            track_data[track_id] = {
                'centroid': (cx, cy),
                'total_distance': 0.0
            }

        # Calculate distance from previous position
        prev_cx, prev_cy = track_data[track_id]['centroid']
        dx = cx - prev_cx
        dy = cy - prev_cy
        dist = np.hypot(dx, dy)

        # Filter unrealistic jumps
        if dist > 30:
            dist = 0

        # Update total distance and centroid
        track_data[track_id]['total_distance'] += dist
        track_data[track_id]['centroid'] = (cx, cy)

        # Display ID and TDP
        #tdp = round(track_data[track_id]['total_distance'], 1)
        #label = f"ID {track_id} | TDP: {tdp}"
        #cv2.putText(frame, label, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        #cv2.rectangle(frame, (xmin, ymin), (xmin + w, ymin + h), (0, 255, 0), 2)

    # Density Detection Using KDTree
    if len(all_centroids) > 1:
        points = np.array(all_centroids)
        tree = spatial.KDTree(points)
        radius = 100  # pixels
        neighbors = tree.query_ball_tree(tree, radius)
        density = np.array([len(n) for n in neighbors])

        index = np.where(density >= 6)[0]

        if len(index) > 0:
            for i in index:
                x, y = points[i]
                cv2.rectangle(frame, (x - 30, y - 30), (x + 30, y + 30), (0, 0, 255), 1)
                cv2.putText(frame, 'High Dense', (x, y - 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 1)

    # Write output frame
    out.write(frame)

    print(f"Processed frame | Time: {time.time() - start_time:.3f}s")


0: 384x640 46 0s, 48.7ms
Speed: 17.6ms preprocess, 48.7ms inference, 374.2ms postprocess per image at shape (1, 3, 384, 640)
Processed frame | Time: 7.399s

0: 384x640 46 0s, 11.0ms
Speed: 2.3ms preprocess, 11.0ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)
Processed frame | Time: 0.038s

0: 384x640 43 0s, 10.8ms
Speed: 2.5ms preprocess, 10.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
Processed frame | Time: 0.027s

0: 384x640 43 0s, 15.1ms
Speed: 2.3ms preprocess, 15.1ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)
Processed frame | Time: 0.038s

0: 384x640 43 0s, 10.8ms
Speed: 1.7ms preprocess, 10.8ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)
Processed frame | Time: 0.026s

0: 384x640 43 0s, 12.1ms
Speed: 2.1ms preprocess, 12.1ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)
Processed frame | Time: 0.028s

0: 384x640 43 0s, 10.8ms
Speed: 1.6ms preprocess, 10.8ms inference, 1.5m

In [13]:
cap.release()
out.release()